In [1]:
!nvidia-smi

Sat Jul 22 09:15:43 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    On   | 00000000:4F:00.0 Off |                    0 |
| 30%   27C    P8    22W / 300W |      3MiB / 45631MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA RTX A6000    On   | 00000000:52:00.0 Off |                    0 |
| 38%   

In [2]:
%env CUDA_VISIBLE_DEVICES=2
%env TOKENIZERS_PARALLELISM=false


env: CUDA_VISIBLE_DEVICES=2
env: TOKENIZERS_PARALLELISM=false


In [3]:
from transformers import XGLMTokenizer, XGLMForCausalLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset, concatenate_datasets
import evaluate
import numpy as np
import torch
from torch import tensor 
import os
from transformers import DataCollatorForLanguageModeling
from functools import partial

In [4]:
torch.manual_seed(0)
torch.use_deterministic_algorithms(True)
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

In [5]:
file_path = "/home/sumire/discourse_context_mt/data/BSD-master/"
data_files = {"train": f"{file_path}train.json", "validation": f"{file_path}dev.json", "test": f"{file_path}test.json"}
dataset = load_dataset("json", data_files=data_files)
dataset

Using custom data configuration default-21a84a9e0ce2914d
Found cached dataset json (/home/sumire/.cache/huggingface/datasets/json/default-21a84a9e0ce2914d/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tag', 'title', 'original_language', 'conversation'],
        num_rows: 670
    })
    validation: Dataset({
        features: ['id', 'tag', 'title', 'original_language', 'conversation'],
        num_rows: 69
    })
    test: Dataset({
        features: ['id', 'tag', 'title', 'original_language', 'conversation'],
        num_rows: 69
    })
})

In [6]:
# define train inputs and targets

inputs = ["Translate English into Japanese: "+sent['en_sentence'] for doc in dataset["train"]["conversation"] for sent in doc]
targets = [sent['ja_sentence'] for doc in dataset["train"]["conversation"] for sent in doc]
#inputs = [inputs = ['Hi this is the systems development department of Company K. = はい、K社システム開発部です。 </s> My name is Takaichi from Company H. = H社の高市と申します。</s> Thank you as always. = いつもお世話になっております。 </s> Thank you as always as well = こちらこそ、お世話になっております。</s> ' + sent['en_sentence'] + " = " for doc in dataset["test"]["conversation"] for sent in doc ][:50] 
inputs = ['Hi this is the systems development department of Company K. = はい、K社システム開発部です。 </s> My name is Takaichi from Company H. = H社の高市と申します。</s> Thank you as always. = いつもお世話になっております。 </s> Thank you as always as well = こちらこそ、お世話になっております。</s> ' 
              + sent['en_sentence'] + ' = ' for doc in dataset["test"]["conversation"] for sent in doc ][:50] 
print (inputs[:5], targets[:5])

['Hi this is the systems development department of Company K. = はい、K社システム開発部です。 </s> My name is Takaichi from Company H. = H社の高市と申します。</s> Thank you as always. = いつもお世話になっております。 </s> Thank you as always as well = こちらこそ、お世話になっております。</s> How is it going, Wayne? = ', "Hi this is the systems development department of Company K. = はい、K社システム開発部です。 </s> My name is Takaichi from Company H. = H社の高市と申します。</s> Thank you as always. = いつもお世話になっております。 </s> Thank you as always as well = こちらこそ、お世話になっております。</s> I'm not too bad. = ", 'Hi this is the systems development department of Company K. = はい、K社システム開発部です。 </s> My name is Takaichi from Company H. = H社の高市と申します。</s> Thank you as always. = いつもお世話になっております。 </s> Thank you as always as well = こちらこそ、お世話になっております。</s> Thank you very much for coming out today. = ', "Hi this is the systems development department of Company K. = はい、K社システム開発部です。 </s> My name is Takaichi from Company H. = H社の高市と申します。</s> Thank you as always. = いつもお世話になっております。 </s> Thank you as a

In [7]:
inputs = [sent['en_sentence'] for doc in dataset["test"]["conversation"] for sent in doc][:50]
[(index+1, sent) for index, sent in enumerate (inputs)]

[(1, 'How is it going, Wayne?'),
 (2, "I'm not too bad."),
 (3, 'Thank you very much for coming out today.'),
 (4, "How's business lately?"),
 (5, "It's been good."),
 (6,
  "We recently commissioned a new facility so I've been busy managing that."),
 (7, 'I read about that on your company website.'),
 (8, 'Congratulations.'),
 (9, 'Thank you.'),
 (10, 'The Japanese market has been very interested in our product.'),
 (11,
  'So having this new facility should satisfy their demand for the next couple of years.'),
 (12, 'The Japanese market is trending upwards lately.'),
 (13,
  'I was in Japan last month and talked to a couple of potential customers.'),
 (14,
  'They all told me that they are looking for a reliable and consistent supply.'),
 (15, 'Well, we are well positioned then.'),
 (16, 'We have 6 facilities with over 1 million ton capacity.'),
 (17, 'You also operate your own terminal.'),
 (18, 'Yes, so we can control the ship loading and schedule.'),
 (19, 'Having control over the

In [8]:
#model_checkpoint = "facebook/xglm-1.7B"
#model_checkpoint = "facebook/xglm-2.9B"
#model_checkpoint = "facebook/xglm-4.5B"
model_checkpoint = "facebook/xglm-7.5B"
#model_checkpoint = "bigscience/mt0-small"
#tokenizer = XGLMTokenizer.from_pretrained(model_checkpoint,padding_side = 'left', max_new_tokens=250, max_length=250, truncation=True, padding='max_length')
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,padding_side = 'left', truncation=True, padding='max_length')
model =  XGLMForCausalLM.from_pretrained(model_checkpoint)
#model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
max_length = 128

def preprocess_function(data): # data should be splitted into train / dev / test internally
    
    #inputs = [sent['en_sentence']  + " = " for doc in data["conversation"] for sent in doc][:50]
    
    
    inputs = ['Hi this is the systems development department of Company K. = はい、K社システム開発部です。 </s> My name is Takaichi from Company H. = H社の高市と申します。</s> Thank you as always. = いつもお世話になっております。 </s> Thank you as always as well = こちらこそ、お世話になっております。</s> ' 
              + sent['en_sentence'] + ' = ' for doc in data["conversation"] for sent in doc ][:50] 
    
    targets = [sent['ja_sentence'] for doc in data["conversation"] for sent in doc][:50]
    
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=500, truncation=True, padding='max_length'
    )
    #model_inputs["labels"]
    
    #model_inputs = tokenizer(inputs,  max_length=250, truncation=True, padding='max_length')
    #print ("input_ids", model_inputs["input_ids"])
    #print ("labels", model_inputs["labels"])
    
    return model_inputs


In [9]:
tokenized_datasets = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset["train"].column_names,
)
# Labels*  2, 6, 48245, 39, 443, 3048, 45469, 45648, 1129, 1338, 30

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [10]:
tokenizer.batch_decode(tokenized_datasets["test"]["labels"], skip_special_tokens=True)

['ウェイン、調子はどうです?',
 'まあまあです。',
 '今日はご足労ありがとう。',
 '景気はどうです?',
 'おかげさまで、順調です。',
 '最近、新しい施設が稼働開始しまして、その管理で忙しくて。',
 'ああ、それ、御社のサイトで読みましたよ。',
 'おめでとうございます。',
 'どうも。',
 '日本市場が当社製品に興味をもっているようです。',
 '新施設のおかげで、今後数年間の需要はえるかと思います。',
 '日本市場は最近上向きだね。',
 '先月日本に出張して、見込みのありそうな数社とミーティングしてきました。',
 '全社、声を揃えて、信頼できて一貫性のある供給を望んでいましたね。',
 'では、当社はいい状況にあるってわけですね。',
 '出力百万トン以上の工場が6件ありますし。',
 '端末も独自に管理されてますしね。',
 'そう、出荷量やスケジュールも管理できます。',
 'サプライチェーンを管理できるのはとても助かります。',
 '柔軟性があるのはとても重要です。',
 '韓国市場は最近どんな状況ですか?',
 '長期の契約は望んでいないと聞きましたが。',
 'そうなんでしょうか?',
 '本当みたいです。',
 '現物契約中心のようです。',
 'どうしてなんでしょう?',
 'サプライヤー一社に傾倒したくないんでしょうね。',
 'なので、いろいろなところから調達したいんですよ。',
 '価格を抑える意味もあるんでしょうか?',
 '価格を比較して、一番安いところから購入するのです。',
 'よっぽど、大変そうですがね。',
 '10年契約をオファーすれば、向こうはのんびりリラックスできるのに。',
 'その方が楽でしょうに。',
 '韓国市場からは目が離せないですね。',
 'そう、最近、いろいろ変化が起きていますからね。',
 '2ヶ月後くらいにまた、情報交換しましょう。',
 'いいですね。',
 '良かった。',
 '今日はお時間をありがとうございました。',
 'こちらこそ。',
 'こんにちは。',
 'A社のライアンです。',
 'こんにちは、B社のジョンといいます。',
 'セミナーはいかがでしたか?',
 '別に普通でしたね。',
 '特に目新しいことは何もなかったです。',
 'この博覧会は初め

In [11]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [12]:
metric1 = evaluate.load("sacrebleu")
metric2 =  evaluate.load("comet")

wmt20-comet-da is already in cache.
Created a temporary directory at /tmp/tmpk3ltbe6p
Writing /tmp/tmpk3ltbe6p/_remote_module_non_scriptable.py
Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaModel: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Encoder model frozen.


In [13]:
def postprocess_text(preds, labels, input_ids):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    input_ids = [[input_id.strip()] for input_id in input_ids]

    return preds, labels, input_ids


In [14]:
tokenizer.encode(" = ")
tokenizer.encode("=")
tokenizer.encode("= ")

[2, 1269, 6]

In [15]:
tgt_lang = "ja"
output_dir = "./results/playingaround"

def compute_metrics(dataset, output_dir, tgt_lang, tokenizer, eval_preds):
    preds, labels, input_ids = eval_preds
    labels = tokenized_datasets["test"]["labels"]  # Why We have to define it again ?? 
    print ("preds before split:", tokenizer.batch_decode(preds[:5], skip_special_tokens=True))
    #print ("labels:", tokenizer.batch_decode(labels[:5], skip_special_tokens=True))
    print ()
    print ("input before split:",tokenizer.batch_decode(input_ids[:5], skip_special_tokens=True))
    print ()
    
    sep = tokenizer.sep_token_id
    split_id = tokenizer.encode("=")[-1]
    
    # Preds
    if isinstance(preds, tuple):
        preds = preds
    #preds = [ np.array_split(item, np.where(item == sep)[-1])[-1]  for item in preds ]
    preds = [ np.array_split(item, np.where(item == split_id)[-1])[-1]  for item in preds ]
    del_index= [0, 1] # Delete "= " in the beggining of the prediction 
    preds =[ np.delete(item, del_index) for item in preds ]
    
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    print ("splited preds: ", decoded_preds[:5])
    print ()
    print ("splitted preds ids: ", preds)
    print ()
    
    # Store prediction inference
    with open(output_dir+'/translations.txt','w', encoding='utf8') as wf:
         for translation, ids in zip(decoded_preds, preds):
            wf.write(translation.strip()+'\n')
            wf.write(str(ids)+'\n')
    
    
    # Labels
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    #print ("labels from eval_preds", [tokenizer.decode(i, skip_special_tokens=True) for i in labels])
    print ()
    #labels= [ np.array_split(item, np.where(item == sep)[-1])[-1]  for item in labels ]
    #print ("checking labels_token:")
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    print ("splitted labels from dataset: ", decoded_labels[:5])
    print ()
    
    # Input_ids
    #input_ids = np.where(input_ids != -100, input_ids, tokenizer.pad_token_id)
    input_ids = [ np.array_split(item, np.where(item == sep)[-1])[-1]  for item in input_ids ]
    input_ids = [ np.array_split(item, np.where(item == split_id)[-1])[0]  for item in input_ids ] 
    decoded_input_ids = tokenizer.batch_decode(input_ids, skip_special_tokens=True)
    print ("splited input_ids", decoded_input_ids[:5])
    print ()
    

    decoded_preds, decoded_labels, decoded_input_ids = postprocess_text(decoded_preds, decoded_labels, decoded_input_ids)
    
    # bleu
    if tgt_lang == "ja":
        bleu = metric1.compute(predictions=decoded_preds, references=decoded_labels, tokenize='ja-mecab')
    else: 
        bleu = metric1.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": bleu["score"]}

    # comet
    #print ("decoded_input_ids:",  [item for decoded_input_id in decoded_input_ids for item in decoded_input_id][:5], "\ndecoded_preds", decoded_preds[:5], "\ndecoded_labels", [item for decoded_label in decoded_labels for item in decoded_label][:5])
    
    comet = metric2.compute(predictions=decoded_preds, references=[item for decoded_label in decoded_labels for item in decoded_label], sources = [item for decoded_input_id in decoded_input_ids for item in decoded_input_id])
    print ("comet")
    print ("decoded_preds", decoded_preds)
    print()
    print ("references", [item for decoded_label in decoded_labels for item in decoded_label])
    print()
    print ("source",  [item for decoded_input_id in decoded_input_ids for item in decoded_input_id])
    result["comet"] =  np.mean(comet["scores"])
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    print(result)

    # Store the score
    with open(output_dir+'/test_score.txt','w', encoding='utf8') as wf:
        for key, value in result.items():
            wf.write(f"{key}: {value}\n") #ensure_ascii=False

    return result


In [16]:

training_args = Seq2SeqTrainingArguments(
    output_dir="./results/playingaround",
    evaluation_strategy="steps",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_eval_batch_size=4, 
    do_predict=True,
    do_eval = True,
    predict_with_generate=True,
    include_inputs_for_metrics=True,
    warmup_steps = 500,
    #metric_for_best_model = "comet"
    #load_best_model_at_end = True,
    greater_is_better = True,
    #save_strategy = "steps",
    eval_delay = 0.0,
    eval_accumulation_steps = 20
    
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=partial(compute_metrics, dataset, output_dir, tgt_lang, tokenizer),
    data_collator=data_collator,
    
)

trainer.evaluate()


***** Running Evaluation *****
  Num examples = 50
  Batch size = 4
You're using a XGLMTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Input length of input_ids is 500, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


Input length of input_ids is 500, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
Input length of input_ids is 500, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
Input length of input_ids is 500, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
Input length of input_ids is 500, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
Input length of input_ids is 500, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
Input length of input_ids is 500, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
Input length of input_ids is 500, but `max_length` is set to 20. This can lead to unexpe

preds before split: ['Hi this is the systems development department of Company K. = はい、K社システム開発部です。  My name is Takaichi from Company H. = H社の高市と申します。 Thank you as always. = いつもお世話になっております。  Thank you as always as well = こちらこそ、お世話になっております。 How is it going, Wayne? = ウ', "Hi this is the systems development department of Company K. = はい、K社システム開発部です。  My name is Takaichi from Company H. = H社の高市と申します。 Thank you as always. = いつもお世話になっております。  Thank you as always as well = こちらこそ、お世話になっております。 I'm not too bad. = そこ", 'Hi this is the systems development department of Company K. = はい、K社システム開発部です。  My name is Takaichi from Company H. = H社の高市と申します。 Thank you as always. = いつもお世話になっております。  Thank you as always as well = こちらこそ、お世話になっております。 Thank you very much for coming out today. = いつも', "Hi this is the systems development department of Company K. = はい、K社システム開発部です。  My name is Takaichi from Company H. = H社の高市と申します。 Thank you as always. = いつもお世話になっております。  Thank you as always as well = こちらこそ、お世話になっております。

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [2]


comet
decoded_preds ['ウ', 'そこ', 'いつも', 'ビジネス', 'いつも', '私たち', '会社', '御', 'ありがとうございます', '日本の', 'こう', '日本の', '去年', '彼', 'はい', '私たち', '貴', 'はい', '流', '柔', '韓国', '聞', '本当', 'はい', '基本的', 'なぜ', '彼', '彼', '価格', '彼', '大変', '彼', 'はい', '市場', 'はい', '次', 'それは', 'はい', '貴', 'いつも', 'はい', '私の', 'はい', '講', 'それは', '彼', '初めて', 'はい', 'それは', 'なに']

references ['ウェイン、調子はどうです?', 'まあまあです。', '今日はご足労ありがとう。', '景気はどうです?', 'おかげさまで、順調です。', '最近、新しい施設が稼働開始しまして、その管理で忙しくて。', 'ああ、それ、御社のサイトで読みましたよ。', 'おめでとうございます。', 'どうも。', '日本市場が当社製品に興味をもっているようです。', '新施設のおかげで、今後数年間の需要はえるかと思います。', '日本市場は最近上向きだね。', '先月日本に出張して、見込みのありそうな数社とミーティングしてきました。', '全社、声を揃えて、信頼できて一貫性のある供給を望んでいましたね。', 'では、当社はいい状況にあるってわけですね。', '出力百万トン以上の工場が6件ありますし。', '端末も独自に管理されてますしね。', 'そう、出荷量やスケジュールも管理できます。', 'サプライチェーンを管理できるのはとても助かります。', '柔軟性があるのはとても重要です。', '韓国市場は最近どんな状況ですか?', '長期の契約は望んでいないと聞きましたが。', 'そうなんでしょうか?', '本当みたいです。', '現物契約中心のようです。', 'どうしてなんでしょう?', 'サプライヤー一社に傾倒したくないんでしょうね。', 'なので、いろいろなところから調達したいんですよ。', '価格を抑える意味もあるんでしょうか?', '価格を比較して、一番安いところから購入するのです。', 'よっぽど、大変

{'eval_loss': 3.142801523208618,
 'eval_bleu': 0.0,
 'eval_comet': -1.054,
 'eval_gen_len': 1.0,
 'eval_runtime': 45.2179,
 'eval_samples_per_second': 1.106,
 'eval_steps_per_second': 0.287}

Findings 
- To translate, specific prompt template is needed: "{Source sentence} = {Target sentence}", otherwise performance  
- zero shot is not working
- From one shot it shows some correct translation (but with only a token)
- We can split the prompt and prediction, also input, by "="

Question 
- Input length of input_ids is 500, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
- Padding and Truncation?
- Labels are not handed over to compute_metrics function, inside the eval_preds (At the moment I directly manually put the reference from dataset)
- Why the model predits only one token ? (But the token is good one)